In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle


In [49]:
import sys
import os

PROJECT_ROOT = os.path.abspath("E:/aleksa_praksa/nlp_internship/data-internship")

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("Project root added:", PROJECT_ROOT)

Project root added: E:\aleksa_praksa\nlp_internship\data-internship


In [ ]:
df = pd.read_csv("E:/aleksa_praksa/nlp_internship/data-internship/data/tokenized_dataset.csv") 

In [51]:
df.head()

,Unnamed: 0,stars,review_body,review_title,language,product_category,review_length,title_length,punctuation_count,all_caps_count,text,preprocessed_text,product_category_star_pred,category_book,category_digital_ebook_purchase,category_other,tokens
0,0,1,Armband ist leider nach 1 Jahr kaputt gegangen,Leider nach 1 Jahr kaputt,de,sports,46,25,0,0,Leider nach 1 Jahr kaputt Armband ist leider n...,Leider nach 1 Jahr kaputt Armband ist leider n...,other,0,0,1,"['Leider', 'nach', '1', 'Jahr', 'kaputt', 'Arm..."
1,1,1,In der Lieferung war nur Ein Akku!,EINS statt ZWEI Akkus!!!,de,home_improvement,34,24,1,0,EINS statt ZWEI Akkus!!! In der Lieferung war ...,EINS statt ZWEI Akkus!!! In der Lieferung war ...,other,0,0,1,"['EINS', 'statt', 'ZWEI', 'Akkus', '!', '!', '..."
2,2,1,"Ein Stern, weil gar keine geht nicht. Es hande...",Achtung Abzocke,de,drugstore,384,15,0,0,"Achtung Abzocke Ein Stern, weil gar keine geht...","Achtung Abzocke Ein Stern, weil gar keine geht...",other,0,0,1,"['Achtung', 'Abzocke', 'Ein', 'Stern', 'weil',..."
3,3,1,"Dachte, das wären einfach etwas festere Binden...",Zu viel des Guten,de,drugstore,205,17,0,0,"Zu viel des Guten Dachte, das wären einfach et...","Zu viel des Guten Dachte, das wären einfach et...",other,0,0,1,"['Zu', 'viel', 'des', 'Guten', 'Dachte', 'das'..."
4,4,1,Meine Kinder haben kaum damit gespielt und nac...,Qualität sehr schlecht,de,toy,114,22,0,0,Qualität sehr schlecht Meine Kinder haben kaum...,Qualität sehr schlecht Meine Kinder haben kaum...,other,0,0,1,"['Qualität', 'sehr', 'schlecht', 'Meine', 'Kin..."


In [52]:
import ast

df['tokens'] = df['tokens'].apply(ast.literal_eval) # str to list

In [53]:
df['text_joined'] = df['tokens'].apply(lambda x: " ".join(x))

In [54]:
def split_language_df(df_one_language):
    return train_test_split(
        df_one_language[['review_length', 'category_book',	'category_digital_ebook_purchase',	'category_other',	'text_joined']],     
        df_one_language['stars'],     
        test_size = 0.2,
        stratify = df_one_language['stars'],
        random_state = 42,

    )

In [55]:
# English
X_train_en, X_test_en, y_train_en, y_test_en = split_language_df(df[df['language'] == 'en'])

In [ ]:
def vectorize(path, X):
    with open(path, "rb") as f:
        vec = pickle.load(f)

    vec_data  = vec.transform(X["text_joined"])

    return vec_data

In [57]:
# English
path_en = r"E:\aleksa_praksa\nlp_internship\data-internship\saved_models\vectorizer\vec_star\tfidf_vec_en.pkl"
tfidf_test_en = vectorize(path_en, X_test_en)
tfidf_train_en = vectorize(path_en, X_train_en)

In [58]:
def load_model(model_pkl_path):
    with open(model_pkl_path, "rb") as f:
        return pickle.load(f)

In [59]:
nb_en = load_model(r"C:\Users\aleksa.cosovic\mlartifacts\889529606865708664\models\m-3763b8d997d543e0af5e09dc1e3075df/artifacts/model.pkl")

lr_en = load_model(r"C:\Users\aleksa.cosovic\mlartifacts\889529606865708664\models\m-dc4651f7f97744a9bc733949e9ab23f0\artifacts\model.pkl")

# does not output probabilities - soft voting requires probabilities.
svm_en_raw = load_model(r"C:\Users\aleksa.cosovic\mlartifacts\889529606865708664\models\m-d980e9f3c4bc450e934cac50dc92a20d\artifacts\model.pkl") 

In [60]:
from sklearn.calibration import CalibratedClassifierCV

svm_en = CalibratedClassifierCV(svm_en_raw, method="sigmoid", cv=3)

svm_en.fit(tfidf_train_en, y_train_en)


,estimator,LinearSVC(C=0.5)
,method,'sigmoid'
,cv,3
,n_jobs,None
,ensemble,'auto'
,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,0.5
,multi_class,'ovr'


In [68]:
import numpy as np

def soft_voting(models, X_test):
    probs = [model.predict_proba(X_test) for model in models]
    avg_probs = np.mean(probs, axis=0)
    return avg_probs.argmax(axis=1) + 1


In [69]:
y_pred_ensemble_en = soft_voting(models = [lr_en, svm_en, nb_en], X_test = tfidf_test_en)


In [70]:
from sklearn.metrics import classification_report, accuracy_score

print("Ensemble soft voting for English reviews with TF-IDF vectorizer")
print(classification_report(y_test_en, y_pred_ensemble_en))
print("Accuracy:", accuracy_score(y_test_en, y_pred_ensemble_en))


Ensemble soft voting for English reviews with TF-IDF vectorizer
              precision    recall  f1-score   support

           1       0.67      0.72      0.69      7961
           2       0.49      0.49      0.49      7968
           3       0.50      0.46      0.48      7960
           4       0.57      0.53      0.55      7973
           5       0.73      0.78      0.75      7971

    accuracy                           0.60     39833
   macro avg       0.59      0.60      0.59     39833
weighted avg       0.59      0.60      0.59     39833

Accuracy: 0.5963397183240027
